In [1]:
import websocket
import talib 
import numpy as np
import json

In [2]:
# Define the socket

cc = 'btcusdt'
interval = '1m'

socket =  f'wss://stream.binance.com:9443/ws/{cc}@kline_{interval}'

In [3]:
#PROVIDING PARAMETERS

core_to_trade = True      # core to trade strategy
core_quantity = 0

aroon_time_period = 14

amount= 1000
core_trade_amount = amount*0.8
trade_amount = amount*0.2

money_end = amount     # keep track of the amount
portfolio = 0          # portfolio quantity  
investments, closes, highs, lows = [], [], [], []       # keep track of investments by updating of remaining amount and closing,highs,lows of candles 
real_time_portfolio_value = []

In [4]:
#PAPER TRADING SIMULATION FUNCTIONS

def buy(allocate_amount,price):
    global money_end, portfolio, investments
    quantity = allocate_amount/price
    money_end -= quantity*price
    portfolio += quantity

    if investments==[]:
        investments.append(allocate_amount)
    
    else:
        investments.append(allocate_amount)
        investments[-1] += investments[-2]

def sell(allocate_amount,price):
    global money_end, portfolio, investments
    quantity = allocate_amount/price
    money_end += quantity*price
    portfolio -= quantity

    investments.append(-allocate_amount)
    investments[-1] += investments[-2]


# price = 100
# buy(300,price)

# print(investments)
# print(portfolio)
# print(money_end)

# price = 150
# sell(200,price)

# print(investments)
# print(portfolio)
# print(money_end)

In [ ]:
# EXTRACT WEBDATA USING WEBSOCKET AND BINANCE

def on_close(ws):
    portfolio_value = portfolio* closes[-1]
    if portfolio_value > 0:
        sell(portfolio_value, price= closes[-1])
    else:
        buy(-portfolio_value, price=closes[-1])
    money_end += investments[-1] 
    print('close')

def on_message(ws, message):
    global closes, highs, lows, core_to_trade, core_quantity, money_end, portfolio, investments, core_quantity
    json_message = json.loads(message)    #JSON for storing and exchanging data
    cs = json_message['k']                # Extract k dictionary from the string
    candle_closed, close, high, low = cs ['x'], cs ['c'], cs ['h'], cs ['l']
    #print(candle_close,close,high,low)

    if candle_closed:
        closes.append(float(close))
        highs.append(float(high))
        lows.append(float(low))
        last_price = closes[-1]
        print(f'Closes: {closes}')
        #print(closes,highs,lows)

        # Setting up core portfolio for 80% of amount (Core Strategy)
        if core_to_trade:         
            buy(core_trade_amount, price = closes[-1])
            print (f'Core Investment : We bought ${core_trade_amount} worth of bitcoin', '\n')
            core_quantity += core_trade_amount / closes[-1]
            core_to_trade = False

        # Setting up aroon startegy for 20% of amount (Satelite Strategy)
        aroon = talib.AROONOSC(np.array(highs),np.array(lows), aroon_time_period)
        last_aroon = round(aroon[-1],2)
        amt = last_aroon * trade_amount/100
        port_value = portfolio*last_price - core_quantity*last_price
        trade_amt = amt - port_value
        print(trade_amt)
        print(port_value)
        
        RT_portfolio_value = port_value + core_quantity*last_price + money_end
        real_time_portfolio_value.append(float(RT_portfolio_value))

        print(f'The last Aroon is "{last_aroon}" and recommended exposure is "${amt}"')
        print(f'Real-Time portfolio value is ${RT_portfolio_value}', '\n')
        print(real_time_portfolio_value)

        if trade_amt > 0:
            buy(trade_amt, price = last_price)
            print(f'we bougnt ${trade_amt} worth of bitcoin', '\n' , '\n' )
        elif trade_amt < 0:
            sell(-trade_amt, price = last_price)
            print(f'we sold ${-trade_amt} worth of bitcoin', '\n' , '\n' )

ws = websocket.WebSocketApp(socket, on_message = on_message, on_close = on_close)



In [6]:
ws.run_forever()

Closes: [27955.56]
Core Investment : We bought $800.0 worth of bitcoin 

nan
0.0
The last Aroon is "nan" and recommended exposure is "$nan"
Real-Time portfolio value is $1000.0 

[1000.0]


KeyboardInterrupt: 

In [ ]:
core_to_trade